In [18]:
from cdp_data import CDPInstances, datasets

seattle_df = datasets.get_session_dataset(
    CDPInstances.Seattle,
    start_datetime="2023-02-01",
    end_datetime="2023-02-07",
    store_transcript=True,
    store_transcript_as_csv=True,
)
seattle_df.sample(2)

Converting transcripts: 100%|██████████| 2/2 [00:00<00:00,  3.01it/s]


,session_datetime,session_index,session_content_hash,video_uri,video_start_time,video_end_time,caption_uri,external_source_id,id,key,event,transcript,transcript_path,transcript_as_csv_path
0,2023-02-01 17:30:00+00:00,0,34a045e7b62ee6a78a40138d0530e92aebd46b741bf7e4...,https://video.seattle.gov/media/council/fin_02...,None,None,https://www.seattlechannel.org/documents/seatt...,None,80cf639b6644,session/80cf639b6644,<cdp_backend.database.models.Event object at 0...,<cdp_backend.database.models.Transcript object...,/Users/shakragoler/VS_Code/cdp/minutes-item-se...,/Users/shakragoler/VS_Code/cdp/minutes-item-se...
1,2023-02-06 22:00:00+00:00,0,90a971243f7ffba7aa29a6ddba812b19de0bdb3aba2859...,https://video.seattle.gov/media/council/brief_...,None,None,https://www.seattlechannel.org/documents/seatt...,None,2f8bd5a37370,session/2f8bd5a37370,<cdp_backend.database.models.Event object at 0...,<cdp_backend.database.models.Transcript object...,/Users/shakragoler/VS_Code/cdp/minutes-item-se...,/Users/shakragoler/VS_Code/cdp/minutes-item-se...


In [23]:
from functools import partial
import pandas as pd

def _process_session(row: pd.Series, extra_meta: dict) -> dict:
    # Read transcript CSV
    transcript = pd.read_csv(row.transcript_as_csv_path)
    
    return {
        "text": " ".join(transcript.text),
        "meta": {
            "event_id": row.event.id,
            "session_id": row.id,
            **extra_meta,
        },
    }

# Create partial with muni name
process_session_seattle = partial(_process_session, extra_meta={"muni": "seattle"})

# Process all rows in dataset to get full text to a single column
processed_sessions = pd.DataFrame(
    list(seattle_df.apply(process_session_seattle, axis=1)), 
)
processed_sessions.sample(1)

,text,meta
1,"≫ Good afternoon, everybody. Today is Monday, ...","{'event_id': '5929a705b6aa', 'session_id': '2f..."


In [24]:
# Store processed dataset to JSONL
processed_sessions.to_json("seattle.jsonl", orient="records", lines=True)